In [ ]:
import numpy as np
import pandas as pd
import time
import csv

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
link = "https://www.dienmayxanh.com/quat/toshiba-f-lsa20-h-vn"

In [ ]:
def create_session_like_driver():
    options = uc.ChromeOptions()

    options.add_argument(
        "--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
    )
    options.add_argument("--lang=vi,en-US;q=0.9,en;q=0.8")

    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = uc.Chrome(options=options)
    return driver

In [ ]:
driver = create_session_like_driver()
driver.get(link)
wait = WebDriverWait(driver, 10)

time.sleep(5)

all_comment = wait.until(EC.element_to_be_clickable((
    By.CSS_SELECTOR,
    'a[class="c-btn-rate btn-view-all"]'
)))
all_comment.click()

time.sleep(2)

batch_size = 6
current_comments = []
comments = []

current_comments = driver.find_elements(
    By.CSS_SELECTOR,
    'p[class="cmt-txt"]'
)

print(f"{len(current_comments)} comments")

for c in current_comments:
    if len(c.text) >= 3:
        comments.append(c.text)

time.sleep(2)

# next_button = wait.until(EC.element_to_be_clickable((
#     By.XPATH,
#     '/html/body/div[8]/div/div[2]/div/div/div[7]/div/div[2]/div/a[5]'
# )))
# next_button.click()
# time.sleep(2)

# next_button = wait.until(EC.element_to_be_clickable((
#     By.XPATH,
#     '/html/body/div[8]/div/div[2]/div/div/div[7]/div/div[2]/div/a[6]'
# )))
page = 2
try:
    next_button = wait.until(EC.element_to_be_clickable((
        By.CSS_SELECTOR,
        f'a[title="trang {page}"]'
    )))
    next_button.click()
    page += 1
    time.sleep(2)
except:
    print("No next button")
    goto: QUIT

while True:
    time.sleep(2)
    current_comments = driver.find_elements(
        By.CSS_SELECTOR,
        'p[class="cmt-txt"]'
    )
    for c in current_comments:
        txt = c.text.strip().replace('\n', ' ')
        if len(txt) >= 3:
            comments.append(txt)
    time.sleep(1)
    print(f"{len(comments)} comments")
    try:
        next_button = wait.until(EC.element_to_be_clickable((
            By.CSS_SELECTOR,
            f'a[title="trang {page}"]'
        )))
        next_button.click()
        page += 1        
    except:
        print("No next button")
        break





QUIT: driver.quit()


In [ ]:
unique_comment = []
set_comment = set()

for item in comments:
    if item not in set_comment:
        unique_comment.append(item)
        set_comment.add(item)

print(len(unique_comment))

In [ ]:
df = pd.DataFrame(unique_comment, columns=["comment"])
df

In [ ]:
df.to_csv(
    "dienmayxanh_comments.csv",
    mode='a',
    index=False,
    header=False,
)